In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
import time
import csv
import pandas as pd

## With this script we extract the 100 Best Sellers from amazon.co.uk(title,author,price)

### Attention!!!As of 19/2/2023 this script works like a charm.There is no quarantee that it will work tomorrow.Websites constantly change and even a small change in the structure of the website can break the script.The good news is that many changes to websites are small and incremental, so we’ll likely be able to update our scraper with only minimal adjustments.

In [3]:
##### Web scrapper that imitates page scrolling for lazy loading pages #####
service = Service(r"C:\Users\cbots\Downloads\MY PORTFOLIO\Amazon Web Scraper Projects\Scraping 100 Best Sellers\chromedriver.exe")
driver = webdriver.Chrome(service=service)
driver.get("https://www.amazon.co.uk/gp/bestsellers/books/ref=pd_zg_ts_books")
time.sleep(2)  # Allow 2 seconds for the web page to open

####  FIRST PAGE  ####
# Scroll down the first page(this part of the code imitates the scrolling of the mouse)
scroll_pause_time = 1 # the code scrolls down the page with a 1 second delay between scrolls so that the page loads properly
screen_height = driver.execute_script("return window.screen.height;")   # get the screen height of the web
i = 1
while True:
    # scroll one screen height each time
    driver.execute_script("window.scrollTo(0, {screen_height}*{i});".format(screen_height=screen_height, i=i))
    i += 1
    time.sleep(scroll_pause_time)
    # update scroll height after every scroll, as the scroll height can change after we scrolled the page
    scroll_height = driver.execute_script("return document.body.scrollHeight;")
    # Break the loop when the height we need to scroll to is larger than the total scroll height
    if screen_height * i > scroll_height:
        break

# Scrape the data from the first page
soup = BeautifulSoup(driver.page_source, "html.parser")
authors1 = soup.findAll(class_=["a-size-small a-link-child", "a-size-small a-color-base"])
page1_authors = [author.get_text(strip=True) for author in authors1]
prices1 = soup.findAll(class_=["_cDEzb_p13n-sc-price_3mJ9Z", "p13n-sc-price"])
page1_prices = [price.get_text(strip=True) for price in prices1]
aTags = soup.find_all('a', {'class': 'a-link-normal'})
page1_titles = []
for tag in aTags:
    # Extract text from div tags with class '_cDEzb_p13n-sc-css-line-clamp-1_1Fn1y'
    divTags1 = tag.find_all('div', {'class': '_cDEzb_p13n-sc-css-line-clamp-1_1Fn1y'})
    for tag in divTags1:
        page1_titles.append(tag.get_text(strip=True))

    # Extract text from div tags with class '_cDEzb_p13n-sc-css-line-clamp-2_EWgCb'
    divTags2 = tag.find_all('div', {'class': '_cDEzb_p13n-sc-css-line-clamp-2_EWgCb'})
    for tag in divTags2:
        page1_titles.append(tag.get_text(strip=True))

# Combine book titles,authors and prices
if len(page1_titles) == len(page1_authors) == len(page1_prices):
    with open('titles_authors_prices.csv', 'w', newline='', encoding='utf-8') as file:
        writer = csv.writer(file)
        for i in range(len(page1_titles)):
            price = page1_prices[i].replace('£', '', 1).strip()
            writer.writerow([f"'{page1_titles[i]}'", page1_authors[i], price])
else:
    print('Error: The number of book titles does not match the number of book authors')

# Pagination-check if there is a Next Page button on the first page
next_button = driver.find_elements(By.CLASS_NAME, "a-last")
if next_button:
    # If there is a next page button, click it
    next_button[0].click()
    time.sleep(3)  # Wait for the second page to load

####  SECOND PAGE  ####
# Scroll down the second page(this part of the code imitates the scrolling of the mouse)
scroll_pause_time = 1 # the code scrolls down the page with a 1 second delay between scrolls so that the page loads properly
screen_height = driver.execute_script("return window.screen.height;")   # get the screen height of the web
i = 1

while True:
    # scroll one screen height each time
    driver.execute_script("window.scrollTo(0, {screen_height}*{i});".format(screen_height=screen_height, i=i))
    i += 1
    time.sleep(scroll_pause_time)
    # update scroll height after every scroll, as the scroll height can change after we scrolled the page
    scroll_height = driver.execute_script("return document.body.scrollHeight;")
    # Break the loop when the height we need to scroll to is larger than the total scroll height
    if screen_height * i > scroll_height:
        break

soup = BeautifulSoup(driver.page_source, "html.parser")
authors2 = soup.findAll(class_=["a-size-small a-link-child", "a-size-small a-color-base"])
page2_authors = [author.get_text(strip=True) for author in authors2]
prices2 = soup.findAll(class_=["_cDEzb_p13n-sc-price_3mJ9Z","p13n-sc-price"])
page2_prices = [price.get_text(strip=True) for price in prices2]
aTags = soup.find_all('a', {'class': 'a-link-normal'})
page2_titles = []
for tag in aTags:
    # Extract text from div tags with class '_cDEzb_p13n-sc-css-line-clamp-1_1Fn1y'
    divTags3 = tag.find_all('div', {'class': '_cDEzb_p13n-sc-css-line-clamp-1_1Fn1y'})
    for tag in divTags3:
        page2_titles.append(tag.get_text(strip=True))

    # Extract text from div tags with class '_cDEzb_p13n-sc-css-line-clamp-2_EWgCb'
    divTags4 = tag.find_all('div', {'class': '_cDEzb_p13n-sc-css-line-clamp-2_EWgCb'})
    for tag in divTags4:
        page2_titles.append(tag.get_text(strip=True))

# Combine book titles,authors and prices
if len(page2_titles) == len(page2_authors) == len(page2_prices):
    with open('titles_authors_prices2.csv', 'w', newline='', encoding='utf-8') as file:
        writer = csv.writer(file)
        for i in range(len(page2_titles)):
            price = page2_prices[i].replace('£', '', 1).strip()
            writer.writerow([f"'{page2_titles[i]}'", page2_authors[i], price])
else:
    print('Error: The number of book titles does not match the number of book authors')


# read the data from the first file
df1 = pd.read_csv('titles_authors_prices.csv', delimiter=',', header=None, names=['Book Title', 'Author', 'Price'])

# read the data from the second file
df2 = pd.read_csv('titles_authors_prices2.csv', delimiter=',', header=None, names=['Book Title', 'Author', 'Price'])

# combine the dataframes into one
df = pd.concat([df1, df2], ignore_index=True)

# write the combined data to a CSV file
df.to_csv('best100_titles_authors_prices.csv', index=False)

driver.quit()

In [4]:
pd.set_option('display.max_rows', None) #display all rows

In [5]:
pd.options.display.max_colwidth=200 # change Book Title column width to fit the whole title

In [6]:
df

,Book Title,Author,Price
0,'Bored of Lunch: The Healthy Air Fryer Book',Nathan Anthony,9.49
1,'Bored of Lunch: The Healthy Slow Cooker Book: THE NUMBER ONE BESTSELLER',Nathan Anthony,9.00
2,'Strong Female Character',Fern Brady,15.63
3,'The World According to Kaleb: THE SUNDAY TIMES BESTSELLER - worldly wisdom from the breakout star of Clarkson’s Farm',Kaleb Cooper,9.00
4,"'It Ends With Us: The emotional #1 Sunday Times bestseller (Lily & Atlas, 1)'",Colleen Hoover,4.50
5,'Atomic Habits: the life-changing million-copy #1 bestseller',James Clear,9.00
6,"'The Ultimate Air Fryer Cookbook: Quick, healthy, energy-saving recipes using UK measurements'",Clare Andrews,10.00
7,"'Thrown: THE SUNDAY TIMES BESTSELLING novel of friendship, heartbreak and pottery for beginners'",Sara Cox,6.74
8,"'Dr Rupy Cooks: Over 100 easy, healthy, flavourful recipes'",Dr Rupy Aujla,11.00
9,'The Keeper of Stories: The most charming and uplifting novel you will read this year!',Sally Page,3.00
